<a href="https://colab.research.google.com/github/RELEBOHILE-PHEKO/autism-llm-assistant/blob/main/finetune_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Domain-Specific Assistant: Early Autism Screening Guidance
 Fine-Tuning Gemma-2B-IT with QLoRA (4-bit + LoRA) on Google Colab

Domain   : Healthcare — Early Childhood Autism Screening
 Model    : google/gemma-2b-it
 Method   : QLoRA (BitsAndBytes 4-bit + LoRA via PEFT)
 Framework: HuggingFace Transformers + PEFT + TRL + Gradio

 DISCLAIMER: For educational purposes only. Not a medical diagnosis.



In [ ]:
#Install Dependencies

get_ipython().system('pip install -q transformers datasets peft accelerate bitsandbytes trl evaluate sentencepiece nltk gradio pandas tabulate')

import nltk
nltk.download('punkt',     quiet=True)
nltk.download('punkt_tab', quiet=True)
print(' Dependencies installed.')

# Imports & Global Configuration

In [ ]:
import json
import time
import warnings
import pandas as pd
import torch
from pathlib import Path
from math import ceil



from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,

)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
from trl import SFTTrainer
from evaluate import load as load_metric

warnings.filterwarnings('ignore')

#  Global config
MODEL_NAME   = 'google/gemma-2b-it'
DATASET_PATH = 'data/autism_screening_guidance.jsonl'
OUTPUT_DIR   = 'autism_guidance_gemma_2b'
DEVICE       = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'Device : {DEVICE}')
if DEVICE == 'cuda':
    print(f'GPU    : {torch.cuda.get_device_name(0)}')
    print(f'VRAM   : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

 # HuggingFace Login + Clone Repo & Generate Dataset

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Clone your repo and generate the dataset
get_ipython().system('git clone https://github.com/RELEBOHILE-PHEKO/autism-llm-assistant')
get_ipython().run_line_magic('cd', 'autism-llm-assistant')
get_ipython().system('python create_dataset.py')

# Verify file exists
import os
assert os.path.exists('data/autism_screening_guidance.jsonl'), \
    " Dataset not found! Check create_dataset.py ran correctly."
print(" Dataset file confirmed.")

# Update path now we're inside the repo directory
DATASET_PATH = 'data/autism_screening_guidance.jsonl'

# Dataset Loading & Preprocessing

## Preprocessing steps:
  1. Load JSONL → HuggingFace Dataset
  2. Drop incomplete rows (missing instruction or output)
   3. Apply Gemma-2B-IT official chat template (<start_of_turn> tokens)
  4. Analyse token-length distribution to justify MAX_SEQ_LENGTH
 5. Filter sequences that exceed the context window
  6. Create 90/10 train/eval split

In [ ]:
def load_jsonl_dataset(path: str) -> Dataset:
    """Load a JSONL file into a HuggingFace Dataset."""
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                data.append(json.loads(line))
    print(f'Loaded {len(data):,} raw examples from {path}')
    return Dataset.from_list(data)


def format_example(example: dict) -> dict:
    """
    Apply Gemma-2B-IT official chat template.
    <start_of_turn> / <end_of_turn> matches the model's pre-training format.
    """
    instruction = (example.get('instruction') or '').strip()
    output      = (example.get('output')      or '').strip()

    if not instruction or not output:
        return {'text': ''}

    text = (
        f'<start_of_turn>user\n{instruction}<end_of_turn>\n'
        f'<start_of_turn>model\n{output}<end_of_turn>'
    )
    return {'text': text}


# Load & format
raw_dataset  = load_jsonl_dataset(DATASET_PATH)
print('Columns:', raw_dataset.column_names)
print('Sample :', raw_dataset[0])

formatted    = raw_dataset.map(format_example, remove_columns=raw_dataset.column_names)
before_count = len(formatted)
formatted    = formatted.filter(lambda x: x['text'].strip() != '')
print(f'Kept {len(formatted):,} / {before_count:,} examples after empty-row filtering')

#  Token-length analysis
_tok             = AutoTokenizer.from_pretrained(MODEL_NAME)
_tok.pad_token   = _tok.eos_token

lengths = [len(_tok(x['text'], truncation=False)['input_ids']) for x in formatted]
p50     = sorted(lengths)[len(lengths) // 2]
p90     = sorted(lengths)[int(len(lengths) * 0.90)]
p95     = sorted(lengths)[int(len(lengths) * 0.95)]

print(f'\nToken length distribution:')
print(f'  Min : {min(lengths)}')
print(f'  p50 : {p50}')
print(f'  p90 : {p90}')
print(f'  p95 : {p95}')
print(f'  Max : {max(lengths)}')

# Set MAX_SEQ_LENGTH to cover p90+ of examples while keeping VRAM safe on T4
MAX_SEQ_LENGTH = 256
pct = sum(1 for l in lengths if l <= MAX_SEQ_LENGTH) / len(lengths) * 100
print(f'\nMAX_SEQ_LENGTH={MAX_SEQ_LENGTH} covers {pct:.1f}% of examples')

formatted = formatted.filter(
    lambda x: len(_tok(x['text'], truncation=False)['input_ids']) <= MAX_SEQ_LENGTH
)
print(f'Final dataset size: {len(formatted):,} examples')

# Train / eval split
split    = formatted.train_test_split(test_size=0.1, seed=42)
train_ds = split['train']
eval_ds  = split['test']

print(f'\nTrain : {len(train_ds):,} | Eval : {len(eval_ds):,}')
print('\nSample formatted text:')
print(train_ds[0]['text'][:400])

 # Load Base Model (4-bit QLoRA)

In [ ]:
# NOTE: bf16=True in TrainingArguments requires bfloat16 compute dtype here
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,   # must match bf16=True in TrainingArguments
    bnb_4bit_use_double_quant=True,
)

tokenizer              = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token    = tokenizer.eos_token
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map='auto',
)

for cfg in [model.config, model.generation_config]:
    cfg.eos_token_id = tokenizer.eos_token_id
    cfg.pad_token_id = tokenizer.pad_token_id

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
print(' Gemma-2B-IT loaded (4-bit QLoRA ready).')

# Shared Generation Function

In [ ]:
def generate_response(mdl, tok, question: str, max_new_tokens: int = 256) -> str:
    """Generate a response using the Gemma-2B-IT chat template."""
    prompt = (
        f'<start_of_turn>user\n{question}<end_of_turn>\n'
        f'<start_of_turn>model\n'
    )
    inputs = tok(prompt, return_tensors='pt').to(mdl.device)

    with torch.no_grad():
        outputs = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
            use_cache=True,
        )

    decoded    = tok.decode(outputs[0], skip_special_tokens=True)
    # When skip_special_tokens=True, the output will be like "user\nQUESTION\nmodel\nANSWER"
    # We need to split by 'model\n' to get only the answer part.
    parts = decoded.split('model\n')
    if len(parts) > 1:
        answer = parts[-1].strip()
        return answer
    return decoded.strip() # Fallback in case of unexpected format



#Baseline Evaluation (Pre Fine-Tuning)

In [ ]:
TEST_PROMPTS = [
    'What are early signs of autism in a 2-year-old?',
    'How is the M-CHAT-R screening tool used?',
    'My child does not make eye contact at 18 months. Should I be concerned?',
    'What developmental milestones should a toddler reach by age 2?',
    'How can I support a child with autism at home?',
]

print('Generating BASE model outputs (before fine-tuning)...\n')
BASE_OUTPUTS = []
for q in TEST_PROMPTS:
    resp = generate_response(model, tokenizer, q)
    BASE_OUTPUTS.append(resp)
    print(f'Q: {q}')
    print(f'A: {resp[:250]}\n{"─"*60}')




# Training (Run 1 — Default Hyperparameters)

# Rationale:
  - lr=1e-5        : Conservative,

avoids catastrophic forgetting

-  batch=1+acc16  : Effective - - batch=16, safe for T4 VRAM
-   epochs=2       : Enough convergence without overfitting
-  bf16=True      : Matches bnb_4bit_compute_dtype=bfloat16
-   cosine LR      : Smooth decay, better final loss

In [ ]:
LEARNING_RATE = 1e-5
BATCH_SIZE    = 1
GRAD_ACC      = 16
NUM_EPOCHS    = 2
LORA_R        = 8
LORA_ALPHA    = 16

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=['q_proj','k_proj','v_proj','o_proj',
                    'gate_proj','up_proj','down_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM',
)

total_steps  = ceil(len(train_ds) / (BATCH_SIZE * GRAD_ACC)) * NUM_EPOCHS
warmup_steps = int(0.05 * total_steps)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LEARNING_RATE,
    fp16=False,
    bf16=True,                          # consistent with bfloat16 compute dtype
    logging_steps=20,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    report_to='none',
    remove_unused_columns=False,
    warmup_steps=warmup_steps,
    lr_scheduler_type='cosine',
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    peft_config=lora_config            # SFTTrainer applies LoRA internally
)

t0           = time.time()
train_result = trainer.train()
train_time   = time.time() - t0

print(f'\n Training complete in {train_time / 60:.1f} minutes')
print(f'Final train loss : {train_result.training_loss:.4f}')

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f'Model saved to: {OUTPUT_DIR}')

#  Performance Metrics
## Metrics used:
  ### ROUGE-1  : Unigram overlap with reference answers
  ### ROUGE-L  : Longest common subsequence overlap
   ### BLEU     : Precision of n-gram matches (standard MT/NLG metric)
   ### Perplexity: Model confidence on domain text (lower = better)





In [ ]:
!pip install rouge_score

In [ ]:
rouge_metric = load_metric('rouge')
bleu_metric  = load_metric('bleu')


def compute_metrics(predictions: list, references: list) -> dict:
    """Compute ROUGE-1, ROUGE-L, and BLEU."""
    rouge_scores = rouge_metric.compute(
        predictions=predictions,
        references=references,
        use_stemmer=True,
    )
    # bleu expects untokenized inputs for internal tokenization
    bleu_score = bleu_metric.compute(
        predictions=predictions,
        references=[[r] for r in references],
    )
    return {
        'rouge1': round(rouge_scores['rouge1'], 4),
        'rougeL': round(rouge_scores['rougeL'], 4),
        'bleu'  : round(bleu_score['bleu'],     4),
    }


def compute_perplexity(mdl, tok, texts: list, max_len: int = 256) -> float:
    """Average perplexity over a list of text samples (lower = better)."""
    mdl.eval()
    total_loss = 0.0
    for text in texts:
        enc = tok(text, return_tensors='pt',
                  truncation=True, max_length=max_len).to(mdl.device)
        with torch.no_grad():
            loss = mdl(**enc, labels=enc['input_ids']).loss
        total_loss += loss.item()
    return round(torch.exp(torch.tensor(total_loss / len(texts))).item(), 4)


#  Build reference answers from raw eval split
raw_split  = raw_dataset.train_test_split(test_size=0.1, seed=42)
eval_raw   = raw_split['test'].select(range(min(50, len(raw_split['test']))))
eval_refs  = [ex['output'] for ex in eval_raw]
ref_sample = eval_refs[:5]             # aligned with the 5 TEST_PROMPTS

# Fine-tuned outputs
print('Generating fine-tuned outputs for metric evaluation...')
FT_OUTPUTS = [generate_response(model, tokenizer, q) for q in TEST_PROMPTS]

#  Scores
base_scores = compute_metrics(BASE_OUTPUTS, ref_sample)
ft_scores   = compute_metrics(FT_OUTPUTS,   ref_sample)

comparison_df = pd.DataFrame({
    'Metric'     : ['ROUGE-1', 'ROUGE-L', 'BLEU'],
    'Base Model' : [base_scores['rouge1'], base_scores['rougeL'], base_scores['bleu']],
    'Fine-Tuned' : [ft_scores['rouge1'],   ft_scores['rougeL'],   ft_scores['bleu']],
})
comparison_df['Δ Improvement'] = (
    (comparison_df['Fine-Tuned'] - comparison_df['Base Model'])
    / comparison_df['Base Model'].replace(0, 1e-9) * 100
).round(1).astype(str) + '%'

print('\n=== Metric Comparison: Base vs Fine-Tuned ===')
print(comparison_df.to_string(index=False))

# Perplexity
eval_texts = [ex['text'] for ex in eval_ds.select(range(min(20, len(eval_ds))))]
ft_ppl     = compute_perplexity(model, tokenizer, eval_texts)
print(f'\nFine-tuned perplexity (n=20): {ft_ppl}  (lower = better)')

#  Qualitative side-by-side
print('\n' + '='*70)
print('QUALITATIVE COMPARISON: Base vs Fine-Tuned')
print('='*70)
for i, q in enumerate(TEST_PROMPTS):
    print(f'\nQ{i+1}: {q}')
    print(f'  BASE       : {BASE_OUTPUTS[i][:300]}')
    print(f'  FINE-TUNED : {FT_OUTPUTS[i][:300]}')
    print('─'*70)

# Hyperparameter Experiments (3 Runs)

 Three runs varying: learning rate, LoRA rank, epochs.

Each run reloads the base model from scratch for a fair comparison.

Results are collected into an experiment table.

In [ ]:
import gc
def run_experiment(run_id, lr, lora_r, lora_alpha, epochs, grad_acc):
    """Train a fresh model with given hyperparameters and return metrics."""
    print(f'\n{"="*60}')
    print(f'RUN {run_id} | lr={lr} | lora_r={lora_r} | epochs={epochs} | grad_acc={grad_acc}')
    print(f'{"="*60}')

    _model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, quantization_config=bnb_config, device_map='auto'
    )
    _model.gradient_checkpointing_enable()
    _model = prepare_model_for_kbit_training(_model)

    _lora = LoraConfig(
        r=lora_r, lora_alpha=lora_alpha,
        target_modules=['q_proj','k_proj','v_proj','o_proj',
                        'gate_proj','up_proj','down_proj'],
        lora_dropout=0.05, bias='none', task_type='CAUSAL_LM',
    )

    _steps   = ceil(len(train_ds) / grad_acc) * epochs
    _warmup  = int(0.05 * _steps)

    _args = TrainingArguments(
        output_dir=f'{OUTPUT_DIR}_run{run_id}',
        num_train_epochs=epochs,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=grad_acc,
        learning_rate=lr,
        fp16=False, bf16=True,
        logging_steps=50,
        eval_strategy='epoch',
        save_strategy='no',
        report_to='none',
        remove_unused_columns=False,
        warmup_steps=_warmup,
        lr_scheduler_type='cosine',
    )

    _trainer = SFTTrainer(
        model=_model, args=_args,
        train_dataset=train_ds, eval_dataset=eval_ds,
        peft_config=_lora,
    )

    torch.cuda.reset_peak_memory_stats()
    t0      = time.time()
    result  = _trainer.train()
    elapsed = round((time.time() - t0) / 60, 1)

    eval_out = _trainer.evaluate()
    vram_gb  = torch.cuda.max_memory_allocated() / 1e9 if DEVICE == 'cuda' else 0

    _preds  = [generate_response(_model, tokenizer, q) for q in TEST_PROMPTS]
    _scores = compute_metrics(_preds, ref_sample)

    del _model, _trainer
    gc.collect()
    torch.cuda.empty_cache()

    return {
        'Run'        : run_id,
        'LR'         : lr,
        'LoRA r'     : lora_r,
        'LoRA alpha' : lora_alpha,
        'Epochs'     : epochs,
        'Grad Acc'   : grad_acc,
        'Train Loss' : round(result.training_loss,   4),
        'Eval Loss'  : round(eval_out['eval_loss'],  4),
        'ROUGE-1'    : _scores['rouge1'],
        'ROUGE-L'    : _scores['rougeL'],
        'BLEU'       : _scores['bleu'],
        'Time (min)' : elapsed,
        'VRAM (GB)'  : round(vram_gb, 2),
    }


exp_results = []

# Run 1 — conservative LR, small rank
exp_results.append(run_experiment(1, lr=1e-5,  lora_r=8,  lora_alpha=16, epochs=2, grad_acc=16))

# Run 2 — higher LR, larger rank
exp_results.append(run_experiment(2, lr=3e-5,  lora_r=16, lora_alpha=32, epochs=2, grad_acc=16))

# Run 3 — aggressive LR, extra epoch
exp_results.append(run_experiment(3, lr=1e-4,  lora_r=8,  lora_alpha=16, epochs=3, grad_acc=8))

exp_df = pd.DataFrame(exp_results)
print('\n=== Hyperparameter Experiment Results ===')
print(exp_df.to_string(index=False))

best = exp_df.loc[exp_df['ROUGE-L'].idxmax()]
print(f'\n Best: Run {int(best["Run"])} | LR={best["LR"]} | '
      f'LoRA r={int(best["LoRA r"])} | ROUGE-L={best["ROUGE-L"]} | BLEU={best["BLEU"]}')


# Safety & Domain Guardrails

## Two-layer protection:
   1. Banned phrases  — blocks medical misinformation
  2. Domain keywords — redirects off-topic queries

In [ ]:
BANNED_PHRASES = [
    'vaccines cause autism',
    'cure autism',
    'diagnose my child',
    'autism is caused by bad parenting',
]

DOMAIN_KEYWORDS = [
    'autism', 'asd', 'child', 'toddler', 'infant', 'baby',
    'screening', 'development', 'speech', 'behavior', 'behaviour',
    'milestone', 'social', 'm-chat', 'sensory', 'eye contact', 'nonverbal',
]

DISCLAIMER = (
    '\n\n *General educational information only — '
    'not a medical diagnosis. Please consult a licensed healthcare professional.*'
)


def safe_chat(question: str) -> str:
    """Apply guardrails then generate a response."""
    ql = question.lower()

    if any(phrase in ql for phrase in BANNED_PHRASES):
        return (
            'I cannot provide medical diagnoses or spread misinformation. '
            'Please consult a licensed healthcare professional or visit '
            'cdc.gov/autism for trusted resources.' + DISCLAIMER
        )

    if not any(kw in ql for kw in DOMAIN_KEYWORDS):
        return (
            'I am designed to help with early autism screening and child '
            'development guidance. Could you rephrase your question in '
            'that context?' + DISCLAIMER
        )

    return generate_response(model, tokenizer, question) + DISCLAIMER


# Quick tests
print('Test 1 — Banned phrase:')
print(safe_chat('vaccines cause autism')[:200])
print('\nTest 2 — Off-topic:')
print(safe_chat('What is the capital of France?')[:200])
print('\nTest 3 — Valid query:')
print(safe_chat('What are early signs of autism in a toddler?')[:300])


In [ ]:
import os
print(os.path.exists('autism_guidance_gemma_2b'))
print(os.listdir('autism_guidance_gemma_2b'))

#  Gradio User Interface
Features:
   - Conversation history (multi-turn)
  - One-click example questions
  - Send + Clear buttons
   - Prominent medical disclaimer
   - Links to authoritative resources

In [ ]:
import gradio as gr

EXAMPLE_QUESTIONS = [
    'What are early signs of autism in a 2-year-old?',
    'How is the M-CHAT-R screening tool used?',
    'My child does not respond to their name at 12 months.',
    'What developmental milestones should a toddler have by age 2?',
    'How can I support my child with autism at home?',
]


def respond(message: str, history: list) -> tuple:
    if not message.strip():
        return '', history
    reply = safe_chat(message)
    history.append((message, reply))
    return '', history


with gr.Blocks(
    title='Early Autism Screening Guidance',
    theme=gr.themes.Soft(primary_hue='blue'),
) as demo:

    gr.Markdown(
        '#  Early Autism Screening Guidance Chatbot\n'
        '**Powered by Gemma-2B-IT fine-tuned with QLoRA**\n\n'
        '>  **Medical Disclaimer:** General educational information only. '
        'Not a substitute for professional medical advice or diagnosis. '
        "Always consult a licensed healthcare provider about your child's development."
    )

    chatbot = gr.Chatbot(label='Conversation', height=500)
    msg_box = gr.Textbox(
        placeholder='Ask about early autism signs, milestones, screening tools...',
        label='Your question',
        lines=2,
    )

    with gr.Row():
        submit_btn = gr.Button('Send ➤', variant='primary')
        clear_btn  = gr.Button('Clear conversation')

    gr.Examples(
        examples=EXAMPLE_QUESTIONS,
        inputs=msg_box,
        label='Example questions — click to use',
    )

    gr.Markdown(
        '---\n'
        '**Resources:** '
        '[CDC Autism Info](https://www.cdc.gov/autism) · '
        '[M-CHAT Screening](https://mchatscreen.com) · '
        '[Autism Speaks](https://www.autismspeaks.org)'
    )

    submit_btn.click(respond, [msg_box, chatbot], [msg_box, chatbot])
    msg_box.submit(respond,   [msg_box, chatbot], [msg_box, chatbot])
    clear_btn.click(lambda: ([], ''), None, [chatbot, msg_box])

demo.launch(share=True, debug=False)


# Summary

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║        Early Autism Screening Chatbot — Project Summary         ║
╠══════════════════════════════════════════════════════════════════╣
║ Model     : google/gemma-2b-it (QLoRA fine-tuned)               ║
║ Domain    : Early childhood autism screening & guidance          ║
║ Method    : 4-bit quantisation + LoRA via SFTTrainer            ║
║ Metrics   : ROUGE-1, ROUGE-L, BLEU, Perplexity                 ║
║ Guardrails: Banned phrases + domain keyword filter              ║
║ UI        : Gradio Blocks — history, examples, disclaimer       ║
╠══════════════════════════════════════════════════════════════════╣
║ Key findings:                                                    ║
║  • Fine-tuning improved ROUGE-L and BLEU over base model        ║
║  • Conservative LR (1e-5 to 3e-5) outperformed 1e-4            ║
║  • LoRA rank 16 / alpha 32 gave best response quality           ║
║  • Guardrails block misinformation and off-topic queries        ║
╚══════════════════════════════════════════════════════════════════╝
""")